In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

train_identity = pd.read_csv('/Users/krc/IEEE-CIS_Fraud_Detection/train_identity.csv')
train_transaction = pd.read_csv('/Users/krc/IEEE-CIS_Fraud_Detection/train_transaction.csv')
test_identity = pd.read_csv('/Users/krc/IEEE-CIS_Fraud_Detection/test_identity.csv')
test_transaction = pd.read_csv('/Users/krc/IEEE-CIS_Fraud_Detection/test_transaction.csv')
test_identity.columns = [col.replace('-','_')for col in test_identity.columns]

train = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')
test = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')

In [3]:
del train_identity,test_identity,train_transaction,test_transaction 

# 전처리 [컬럼drop(null_80%이상)]

In [4]:
dff_column_to_drop = train.columns[(train.isna().sum() / len(train) > 0.2)]

In [5]:
dff_column_to_drop=dff_column_to_drop.drop(['id_30','id_31','id_32','id_33','id_34','DeviceType','DeviceInfo'])

In [6]:
def preprocess(df) :
    df = df.drop(columns=dff_column_to_drop).copy()
    # object 범주형 자료 결측치 - 최빈값으로 채워 넣기
    # df.ProductCD.value_counts().index # ['W', 'C', 'R', 'H', 'S']
    df.ProductCD.fillna(df.ProductCD.mode(), inplace=True)
    from sklearn.preprocessing import LabelEncoder
    lbl_enc = LabelEncoder()
    lbl_enc.fit(df[['ProductCD']])
    df['ProductCD'] = lbl_enc.transform(df[['ProductCD']])
    # df.card4.value_counts().index # ['visa', 'mastercard', 'american express', 'discover']
    df.card4.fillna(df.card4.mode(), inplace=True)
    lbl_enc.fit(df[['card4']])
    df['card4'] = lbl_enc.transform(df[['card4']])
    # df.card6.value_counts().index # ['debit', 'credit', 'debit or credit', 'charge card']
    df.card6.fillna(df.card6.mode(), inplace=True)
    lbl_enc.fit(df[['card6']])
    df['card6'] = lbl_enc.transform(df[['card6']])
    # df.P_emaildomain.value_counts().index    # [ gmail yahoo etc   ]
    df.P_emaildomain.fillna(df.P_emaildomain.mode(), inplace=True)
    lbl_enc.fit(df[['P_emaildomain']])
    df['P_emaildomain'] = lbl_enc.transform(df[['P_emaildomain']])
    # id_30
    df.id_30.fillna(df.id_30.mode(), inplace=True)
    lbl_enc.fit(df[['id_30']])
    df['id_30'] = lbl_enc.transform(df[['id_30']])
    #id_31
    df.id_31.fillna(df.id_31.mode(), inplace=True)
    lbl_enc.fit(df[['id_31']])
    df['id_31'] = lbl_enc.transform(df[['id_31']])
    #id_33
    df.id_33.fillna(df.id_33.mode(), inplace=True)
    lbl_enc.fit(df[['id_33']])
    df['id_33'] = lbl_enc.transform(df[['id_33']])
    #id_34
    df.id_34.fillna(df.id_34.mode(), inplace=True)
    lbl_enc.fit(df[['id_34']])
    df['id_34'] = lbl_enc.transform(df[['id_34']])
    #DeviceType
    df.DeviceType.fillna(df.DeviceType.mode(), inplace=True)
    lbl_enc.fit(df[['DeviceType']])
    df['DeviceType'] = lbl_enc.transform(df[['DeviceType']])
    #DeviceInfo
    df.DeviceInfo.fillna(df.DeviceInfo.mode(), inplace=True)
    lbl_enc.fit(df[['DeviceInfo']])
    df['DeviceInfo'] = lbl_enc.transform(df[['DeviceInfo']])
    

    #NULL값이 있는지 한눈에 보기
    display(np.sum(df.isnull() ).to_frame().transpose() )
    display(f'지금 {np.sum(np.sum(df.isnull()))} 개의 null값이 있습니다.' )


    # continuous 연속형 자료 결측치 추가
    # 방법 1 - 중앙값으로 채워 넣기
    temp_list_1 = df.columns[df.isnull().sum() > 0]  # 결측치가 있는 컬럼을 골라서 리스트에 넣기
    for aa in temp_list_1 :
        df[aa].fillna(df[aa].median(), inplace=True)  # 중앙값으로 채워 넣기
    # df.info(verbose=True, show_counts=True)

    print("Null :", df.isnull().sum().sum() )

    return df

In [7]:
train = preprocess(train)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V319,V320,V321,id_30,id_31,id_32,id_33,id_34,DeviceType,DeviceInfo
0,0,0,0,0,0,0,8933,1565,0,4259,...,12,12,12,0,0,512954,0,0,0,0


'지금 6082443 개의 null값이 있습니다.'

Null : 0


In [8]:
test = preprocess(test)

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,V319,V320,V321,id_30,id_31,id_32,id_33,id_34,DeviceType,DeviceInfo
0,0,0,0,0,0,8654,3002,0,4547,0,...,3,3,3,0,0,436020,0,0,0,0


'지금 1500255 개의 null값이 있습니다.'

Null : 0


In [9]:
y = train['isFraud']
x = train.drop('isFraud',axis=1)

In [10]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x,y, test_size=0.2, random_state=0xC0FFEE)  # 42, 0xC0FFEE

In [11]:
#결측치만 채운 데이터 성능 확인
from sklearn.metrics import accuracy_score, precision_score,recall_score,confusion_matrix

def get_clf_eval(y_test,pred):
    confusion = confusion_matrix(y_test,pred)
    accuracy = accuracy_score(y_test,pred)
    precision  = precision_score(y_test,pred)
    recall = recall_score(y_test,pred)
    print('오차 행렬')
    print(confusion)
    print('정확도:{0:.4f}, 정밀도:{1:.4f}, 재현율:{2:.4f}'.format(accuracy,precision,recall))

In [ ]:
def get_model_train_eval(model,ftr_train=None,ftr_test=None,tgt_train=None,tgt_test=None):
    model.fit(ftr_train,tgt_train)
    pred = model.predict(ftr_test)
    pred_proba = model.predict_proba(ftr_test)[:,1]
    get_clf_eval(tgt_test,pred)

In [ ]:
from lightgbm import LGBMClassifier

lgbm_clf = LGBMClassifier(n_estimators=1000,num_leaves=64,n_jobs=-1,boost_from_average=False)
get_model_train_eval(lgbm_clf,ftr_train=x_train,ftr_test=x_val,tgt_train=y_train,tgt_test=y_val)

# feature engineering

In [12]:
#transactionAmt
# 클립하여 0부터 5000사이의 값으로 봅니다
train['TransactionAmt'] = train['TransactionAmt'].clip(0,5000)
test['TransactionAmt']  = test['TransactionAmt'].clip(0,5000)

# 거래 액수가 일반적인지 아닌지를 봅니다
train['TransactionAmt_check'] = np.where(train['TransactionAmt'].isin(test['TransactionAmt']), 1, 0)
test['TransactionAmt_check']  = np.where(test['TransactionAmt'].isin(train['TransactionAmt']), 1, 0)

print('train')
print(train.TransactionAmt_check.value_counts())
print(' ')
print('test')
print(test.TransactionAmt_check.value_counts())

train
1    527684
0     62856
Name: TransactionAmt_check, dtype: int64
 
test
1    490910
0     15781
Name: TransactionAmt_check, dtype: int64


In [13]:
# v's pca 
from sklearn.preprocessing import StandardScaler

v_train = train.loc[:,'V12':'V321']
v_train = v_train.fillna(-999)
v_train = StandardScaler().fit_transform(v_train)
v_test = test.loc[:,'V12':'V321']
v_test = v_test.fillna(-999)
v_test = StandardScaler().fit_transform(v_test)


columns = ['PCA_V1','PCA_V2','PCA_V3','PCA_V4','PCA_V5','PCA_V6','PCA_V7','PCA_V8','PCA_V9','PCA_V10','PCA_V11','PCA_V12','PCA_V13','PCA_V14','PCA_V15','PCA_V16','PCA_V17','PCA_V18','PCA_V19','PCA_V20','PCA_V21','PCA_V22','PCA_V23','PCA_V24','PCA_V25','PCA_V26','PCA_V27','PCA_V28','PCA_V29','PCA_V30']
from sklearn.decomposition import PCA

pca = PCA(n_components=30)
principalComponents_train = pca.fit_transform(v_train)
principalDF_train = pd.DataFrame(data=principalComponents_train,columns=columns)
principalComponents_test = pca.fit_transform(v_test)
principalDF_test = pd.DataFrame(data=principalComponents_test,columns=columns)

train = train.drop(train.loc[:,'V12':'V321'].columns,axis=1)
test = test.drop(test.loc[:,'V12':'V321'].columns,axis=1)

train = pd.concat([train,principalDF_train],axis=1)
test = pd.concat([test,principalDF_test],axis=1)

# over sampling

In [14]:
#feature engineering 한 데이터 셋에 over sampling 해주기
y = train['isFraud']
x = train.drop('isFraud',axis=1)

In [15]:
from imblearn.over_sampling import SMOTE

def smote(x,y):
    smote = SMOTE(random_state=0)
    x_over ,y_over = smote.fit_resample(x,y)
    print('smote 적용 후 레이블 값 분포:\n',pd.Series(y_over).value_counts())
    return x_over, y_over

In [16]:
x_over ,y_over = smote(x,y)

smote 적용 후 레이블 값 분포:
 0    569877
1    569877
Name: isFraud, dtype: int64


# column 조합으로 주요 식별 정보 만들기

In [17]:
import datetime
def date_columns(all_data):
    START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')
    all_data['DT_time'] = all_data['TransactionDT'].apply(lambda x: (START_DATE + datetime.timedelta(seconds = x))).astype('object')
    all_data['diff_days'] = all_data['D1'].fillna(0).apply(lambda x: (datetime.timedelta(days = x))).astype('object')
    all_data['client_startdate_date'] = (all_data['DT_time']- all_data['diff_days']).astype('object')
    all_data['client_startdate'] = (all_data['DT_time']- all_data['diff_days']).apply(lambda x:str(x.date())).astype('object')

In [18]:
date_columns(x_over)
date_columns(test)

In [19]:
import hashlib
def card_info_hash(x):
    s = (str(int(x['card1']))+
         str(int(x['card2']))+
         str(int(x['card3']))+
         str(x['card4'])+
         str(int(x['card5']))+
         str(x['card6']))
    h = hashlib.sha256(s.encode('utf-8')).hexdigest()[0:15]
    return h

In [20]:
import hashlib
def device_hash(x):
    s =  str(x['id_30'])+str(x['id_31'])+str(x['id_32'])+str(x['id_33'])+str( x['DeviceType'])+ str(x['DeviceInfo'])
    h = hashlib.sha256(s.encode('utf-8')).hexdigest()[0:15]
    return h

In [21]:
x_over['card_hash'] = x_over.apply(lambda x: card_info_hash(x), axis=1)

In [22]:
test['card_hash'] = test.apply(lambda x: card_info_hash(x), axis=1)

In [23]:
x_over['device_hash'] = x_over.apply(lambda x: device_hash(x), axis=1)

In [24]:
test['device_hash'] = test.apply(lambda x: device_hash(x), axis=1)

In [25]:
#label encoding
from sklearn import preprocessing
def label_encoding(df):
    for f in df.columns:
        if df[f].dtype == 'object':
            le = preprocessing.LabelEncoder()
            le.fit(list(df[f].values))
            df[f] = le.transform(list(df[f].values))
        else :
            pass

In [26]:
label_encoding(x_over)
label_encoding(test)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_over,y_over, test_size=0.2, random_state=0xC0FFEE)  # 42, 0xC0FFEE

In [ ]:
#over sampling 한 데이터 셋 성능 보기
from lightgbm import LGBMClassifier

lgbm_clf = LGBMClassifier(n_estimators=1000,num_leaves=64,n_jobs=-1,boost_from_average=False)
get_model_train_eval(lgbm_clf,ftr_train=x_train,ftr_test=x_val,tgt_train=y_train,tgt_test=y_val)

# 필요 없는 columns 제거

In [27]:
drop_columns = ['TransactionDT','card1', 'card2', 'card3', 'card4', 'card5', 'card6','id_30', 'id_31',
       'id_32', 'id_33', 'id_34', 'DeviceType', 'DeviceInfo','DT_time', 'diff_days', 'client_startdate_date']

In [28]:
x_over = x_over.drop(drop_columns,axis=1)

In [29]:
test = test.drop(drop_columns,axis=1)

# HyperOpt를 이용한 하이퍼 파라미터 최적화

1.xgboost 

In [ ]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from hyperopt import STATUS_OK

def objective_func(search_space):
    xgb_clf = XGBClassifier(n_estimators=100, max_depth=int(search_space['max_depth']),
                           min_child_weight=int(search_space['min_child_weight']),
                           learning_rate=search_space['learning_rate'],
                           colsample_bytree = search_space['colsample_bytree'],
                           eval_metric ='logloss')
    accuracy = cross_val_score(xgb_clf,x_over,y_over,scoring='accuracy',cv=3)
    
    return {'loss':-1*np.mean(accuracy),'status':STATUS_OK}

In [ ]:
from hyperopt import hp
xgb_search_space = {
    'max_depth':hp.quniform('max_depth',5,20,1),
    'min_child_weight':hp.quniform('min_child_weight',1,2,1),
    'learning_rate':hp.uniform('learning_rate',0.01,0.2),
    'colsample_bytree':hp.uniform('colsample_bytree',0.5,1)
}

In [ ]:
from hyperopt import fmin,tpe,Trials

trial_val = Trials()
best = fmin(
    fn=objective_func,
    space = xgb_search_space,
    algo = tpe.suggest,
    max_evals = 50,
    trials = trial_val, rstate=np.random.default_rng(seed=9)
)
print('best:',best)

2.lightgbm

In [ ]:
lgbm_search_space={
    'num_leaves':hp.quniform('num_leaves',32,64,1),
    'max_depth':hp.quniform('max_depth',100,160,1),
    'min_child_samples':hp.quniform('min_child_samples',60,100,1),
    'subsample':hp.uniform('subsample',0.7,1),
    'learning_rate':hp.uniform('learning_rate',0.01,0.2)
    
}

In [ ]:
from sklearn.model_selection import cross_val_score
def objective_func(search_space):
    lgbm_clf = LGBMClassifier(n_estimators=100,
                             num_leaves=int(search_space['num_leaves']),
                             max_depth = int(search_space['min_child_samples']),
                             subsample = search_space['subsample'],
                             learning_rate=search_space['learning_rate'])
    accuracy = cross_val_score(lgbm_clf,x_over,y_over,scoring='accuracy',cv=3)
    
    return {'loss':-1*np.mean(accuracy),'status':STATUS_OK}

In [ ]:
from hyperopt import fmin,tpe,Trials

trials=Trials()

best_1 = fmin(fn=objective_func,space=lgbm_search_space,algo=tpe.suggest,
           max_evals=50,trials=trials, rstate=np.random.default_rng(seed=30))
print('best:',best_1)

3.catboost

In [ ]:
cat_search_space={
        'depth': hp.quniform("depth", 1, 6, 1),
        'border_count': hp.uniform ('border_count', 32, 255),
        'learning_rate': hp.loguniform('learning_rate', -5.0, -2),
        'l2_leaf_reg': hp.uniform('l2_leaf_reg', 3, 8),
}

In [ ]:
from catboost import CatBoostClassifier
def objective_func(search_space):
    cat_clf = CatBoostClassifier(n_estimators=100,
                                learning_rate=search_space['learning_rate'],
                                depth=int(search_space['depth']),
                                eval_metric='AUC',
                                l2_leaf_reg=search_space['l2_leaf_reg'],
                                early_stopping_rounds=50,
                                border_count=int(search_space['border_count']),
                                verbose=False
                                        )
    accuracy = cross_val_score(cat_clf,x_over,y_over,scoring='accuracy',cv=3)
    
    return {'loss':-1*np.mean(accuracy),'status':STATUS_OK}
    

In [ ]:
trials = Trials()
best_2 = fmin(
        fn=objective_func,
        space=cat_search_space,
        algo=tpe.suggest,
        max_evals=50,
        trials=trials,
        )
print('best:',best_2)

# CV 세트 기반의 스태킹

In [30]:
#lgbm 최적 하이퍼 파라미터 이용
'''best: {'learning_rate': 0.02405400245706958, 'max_depth': 128.0,
'min_child_samples': 75.0, 'num_leaves': 40.0, 'subsample': 0.9280237438901546}
'''
from lightgbm import LGBMClassifier

lgbm_wrapper = LGBMClassifier(
    n_estimators = 400,
    learning_rate = 0.02405400245706958,
    max_dept= 128,
    min_child_samples= 75,
    num_leaves = 40,
    subsample = 0.9280237438901546,
)

In [32]:
#catboost 최적 하이퍼 파라미터 이용
'''best: {'border_count': 174.29373200360376, 'depth': 1.0, 
'l2_leaf_reg': 5.0425980576236995, 'learning_rate': 0.027929292448935134}'''
from catboost import CatBoostClassifier
cat_wrapper = CatBoostClassifier(
    n_estimators = 400,
    border_count = 174,
    depth = 1,
    l2_leaf_reg = 5.0425980576236995,
    learning_rate = 0.027929292448935134
)

In [33]:
x_over = np.array(x_over)
y_over = np.array(y_over)
test = np.array(test)

In [34]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

# 개별 기반 모델에서 최종 메타 모델이 사용할 학습 및 테스트용 데이터를 생성하기 위한 함수. 
def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds ):
    # 지정된 n_folds값으로 KFold 생성.
    kf = KFold(n_splits=n_folds, shuffle=False)
    #추후에 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화 
    train_fold_pred = np.zeros((X_train_n.shape[0] ,1 ))
    test_pred = np.zeros((X_test_n.shape[0],n_folds))
    print(model.__class__.__name__ , ' model 시작 ')
    
    for folder_counter , (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        #입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 셋 추출 
        print('\t 폴드 세트: ',folder_counter,' 시작 ')
        X_tr = X_train_n[train_index] 
        y_tr = y_train_n[train_index] 
        X_te = X_train_n[valid_index]  
        
        #폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행.
        model.fit(X_tr , y_tr)       
        #폴드 세트 내부에서 다시 만들어진 검증 데이터로 기반 모델 예측 후 데이터 저장.
        train_fold_pred[valid_index, :] = model.predict_proba(X_te)[:,1].reshape(-1,1)
        #입력된 원본 테스트 데이터를 폴드 세트내 학습된 기반 모델에서 예측 후 데이터 저장. 
        test_pred[:, folder_counter] = model.predict_proba(X_test_n)[:,1]
            
    # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성 
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)    
    
    #train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean은 테스트 데이터
    return train_fold_pred , test_pred_mean

In [35]:
cat_train, cat_test = get_stacking_base_datasets(cat_wrapper, x_over, y_over, test, 7)
#xgb_train, xgb_test = get_stacking_base_datasets(xgb_wrapper, x_over, y_over, test, 7)
lgbm_train, lgbm_test = get_stacking_base_datasets(lgbm_wrapper, x_over, y_over, test,  7)    

CatBoostClassifier  model 시작 
	 폴드 세트:  0  시작 
0:	learn: 0.6825094	total: 152ms	remaining: 1m
1:	learn: 0.6725016	total: 223ms	remaining: 44.3s
2:	learn: 0.6630939	total: 290ms	remaining: 38.3s
3:	learn: 0.6542604	total: 361ms	remaining: 35.7s
4:	learn: 0.6459598	total: 437ms	remaining: 34.5s
5:	learn: 0.6386862	total: 507ms	remaining: 33.3s
6:	learn: 0.6311349	total: 577ms	remaining: 32.4s
7:	learn: 0.6245129	total: 641ms	remaining: 31.4s
8:	learn: 0.6176421	total: 712ms	remaining: 30.9s
9:	learn: 0.6116114	total: 792ms	remaining: 30.9s
10:	learn: 0.6053549	total: 862ms	remaining: 30.5s
11:	learn: 0.5998546	total: 931ms	remaining: 30.1s
12:	learn: 0.5941541	total: 993ms	remaining: 29.6s
13:	learn: 0.5891377	total: 1.07s	remaining: 29.6s
14:	learn: 0.5839351	total: 1.14s	remaining: 29.4s
15:	learn: 0.5793530	total: 1.22s	remaining: 29.2s
16:	learn: 0.5746130	total: 1.29s	remaining: 29.1s
17:	learn: 0.5704173	total: 1.36s	remaining: 29s
18:	learn: 0.5660142	total: 1.44s	remaining: 28.8s

162:	learn: 0.3808864	total: 12.3s	remaining: 17.8s
163:	learn: 0.3803939	total: 12.3s	remaining: 17.7s
164:	learn: 0.3799704	total: 12.4s	remaining: 17.7s
165:	learn: 0.3761590	total: 12.5s	remaining: 17.6s
166:	learn: 0.3756888	total: 12.6s	remaining: 17.5s
167:	learn: 0.3750788	total: 12.6s	remaining: 17.4s
168:	learn: 0.3746557	total: 12.7s	remaining: 17.4s
169:	learn: 0.3743492	total: 12.8s	remaining: 17.3s
170:	learn: 0.3739021	total: 12.8s	remaining: 17.2s
171:	learn: 0.3734849	total: 12.9s	remaining: 17.1s
172:	learn: 0.3730935	total: 13s	remaining: 17s
173:	learn: 0.3725259	total: 13.1s	remaining: 17s
174:	learn: 0.3721204	total: 13.1s	remaining: 16.9s
175:	learn: 0.3718285	total: 13.2s	remaining: 16.8s
176:	learn: 0.3713996	total: 13.3s	remaining: 16.7s
177:	learn: 0.3710036	total: 13.4s	remaining: 16.7s
178:	learn: 0.3705827	total: 13.4s	remaining: 16.6s
179:	learn: 0.3701984	total: 13.5s	remaining: 16.5s
180:	learn: 0.3698160	total: 13.6s	remaining: 16.4s
181:	learn: 0.3694

324:	learn: 0.3279487	total: 21.8s	remaining: 5.03s
325:	learn: 0.3277525	total: 21.9s	remaining: 4.96s
326:	learn: 0.3274856	total: 21.9s	remaining: 4.89s
327:	learn: 0.3273011	total: 22s	remaining: 4.82s
328:	learn: 0.3271289	total: 22s	remaining: 4.75s
329:	learn: 0.3269998	total: 22.1s	remaining: 4.68s
330:	learn: 0.3268222	total: 22.1s	remaining: 4.61s
331:	learn: 0.3266380	total: 22.1s	remaining: 4.54s
332:	learn: 0.3264562	total: 22.2s	remaining: 4.46s
333:	learn: 0.3260123	total: 22.2s	remaining: 4.4s
334:	learn: 0.3258263	total: 22.3s	remaining: 4.33s
335:	learn: 0.3255503	total: 22.4s	remaining: 4.26s
336:	learn: 0.3253717	total: 22.4s	remaining: 4.19s
337:	learn: 0.3251930	total: 22.5s	remaining: 4.12s
338:	learn: 0.3250153	total: 22.5s	remaining: 4.05s
339:	learn: 0.3248501	total: 22.6s	remaining: 3.98s
340:	learn: 0.3246874	total: 22.6s	remaining: 3.91s
341:	learn: 0.3245172	total: 22.7s	remaining: 3.84s
342:	learn: 0.3243403	total: 22.7s	remaining: 3.77s
343:	learn: 0.324

84:	learn: 0.4599482	total: 6.03s	remaining: 22.3s
85:	learn: 0.4586687	total: 6.1s	remaining: 22.3s
86:	learn: 0.4577240	total: 6.17s	remaining: 22.2s
87:	learn: 0.4565144	total: 6.25s	remaining: 22.2s
88:	learn: 0.4556134	total: 6.32s	remaining: 22.1s
89:	learn: 0.4546385	total: 6.4s	remaining: 22s
90:	learn: 0.4537368	total: 6.47s	remaining: 22s
91:	learn: 0.4528708	total: 6.54s	remaining: 21.9s
92:	learn: 0.4520296	total: 6.62s	remaining: 21.8s
93:	learn: 0.4508937	total: 6.69s	remaining: 21.8s
94:	learn: 0.4500557	total: 6.75s	remaining: 21.7s
95:	learn: 0.4491881	total: 6.82s	remaining: 21.6s
96:	learn: 0.4483722	total: 6.89s	remaining: 21.5s
97:	learn: 0.4475439	total: 6.96s	remaining: 21.5s
98:	learn: 0.4467466	total: 7.03s	remaining: 21.4s
99:	learn: 0.4456822	total: 7.11s	remaining: 21.3s
100:	learn: 0.4448915	total: 7.18s	remaining: 21.3s
101:	learn: 0.4441303	total: 7.25s	remaining: 21.2s
102:	learn: 0.4433608	total: 7.32s	remaining: 21.1s
103:	learn: 0.4425521	total: 7.39s

244:	learn: 0.3766376	total: 17.7s	remaining: 11.2s
245:	learn: 0.3763432	total: 17.7s	remaining: 11.1s
246:	learn: 0.3760489	total: 17.8s	remaining: 11s
247:	learn: 0.3757449	total: 17.9s	remaining: 11s
248:	learn: 0.3754370	total: 18s	remaining: 10.9s
249:	learn: 0.3751395	total: 18s	remaining: 10.8s
250:	learn: 0.3748407	total: 18.1s	remaining: 10.7s
251:	learn: 0.3745372	total: 18.2s	remaining: 10.7s
252:	learn: 0.3742469	total: 18.2s	remaining: 10.6s
253:	learn: 0.3740203	total: 18.3s	remaining: 10.5s
254:	learn: 0.3737008	total: 18.4s	remaining: 10.5s
255:	learn: 0.3734006	total: 18.5s	remaining: 10.4s
256:	learn: 0.3731151	total: 18.6s	remaining: 10.3s
257:	learn: 0.3728242	total: 18.6s	remaining: 10.3s
258:	learn: 0.3724138	total: 18.7s	remaining: 10.2s
259:	learn: 0.3721356	total: 18.8s	remaining: 10.1s
260:	learn: 0.3718485	total: 18.8s	remaining: 10s
261:	learn: 0.3714266	total: 18.9s	remaining: 9.97s
262:	learn: 0.3712098	total: 19s	remaining: 9.9s
263:	learn: 0.3709351	tot

...

91:	learn: 0.4504548	total: 5.74s	remaining: 19.2s
92:	learn: 0.4495907	total: 5.81s	remaining: 19.2s
93:	learn: 0.4487479	total: 5.88s	remaining: 19.1s
94:	learn: 0.4478265	total: 5.95s	remaining: 19.1s
95:	learn: 0.4467581	total: 6.02s	remaining: 19.1s
96:	learn: 0.4458697	total: 6.09s	remaining: 19s
97:	learn: 0.4450635	total: 6.17s	remaining: 19s
98:	learn: 0.4442619	total: 6.24s	remaining: 19s
99:	learn: 0.4433954	total: 6.31s	remaining: 18.9s
100:	learn: 0.4426314	total: 6.38s	remaining: 18.9s
101:	learn: 0.4418158	total: 6.45s	remaining: 18.8s
102:	learn: 0.4410300	total: 6.52s	remaining: 18.8s
103:	learn: 0.4402131	total: 6.6s	remaining: 18.8s
104:	learn: 0.4392273	total: 6.67s	remaining: 18.7s
105:	learn: 0.4384842	total: 6.73s	remaining: 18.7s
106:	learn: 0.4377708	total: 6.8s	remaining: 18.6s
107:	learn: 0.4370517	total: 6.87s	remaining: 18.6s
108:	learn: 0.4362404	total: 6.94s	remaining: 18.5s
109:	learn: 0.4355062	total: 7.01s	remaining: 18.5s
110:	learn: 0.4348158	total: 

250:	learn: 0.3737224	total: 17.6s	remaining: 10.5s
251:	learn: 0.3734571	total: 17.7s	remaining: 10.4s
252:	learn: 0.3731634	total: 17.8s	remaining: 10.3s
253:	learn: 0.3728881	total: 17.9s	remaining: 10.3s
254:	learn: 0.3726168	total: 17.9s	remaining: 10.2s
255:	learn: 0.3723241	total: 18s	remaining: 10.1s
256:	learn: 0.3720383	total: 18.1s	remaining: 10.1s
257:	learn: 0.3717854	total: 18.2s	remaining: 9.99s
258:	learn: 0.3713844	total: 18.2s	remaining: 9.93s
259:	learn: 0.3711326	total: 18.3s	remaining: 9.86s
260:	learn: 0.3708785	total: 18.4s	remaining: 9.79s
261:	learn: 0.3706056	total: 18.5s	remaining: 9.73s
262:	learn: 0.3703568	total: 18.5s	remaining: 9.66s
263:	learn: 0.3700705	total: 18.6s	remaining: 9.59s
264:	learn: 0.3697987	total: 18.7s	remaining: 9.52s
265:	learn: 0.3695427	total: 18.8s	remaining: 9.46s
266:	learn: 0.3692968	total: 18.9s	remaining: 9.39s
267:	learn: 0.3690526	total: 18.9s	remaining: 9.32s
268:	learn: 0.3688428	total: 19s	remaining: 9.25s
269:	learn: 0.36

13:	learn: 0.6018964	total: 702ms	remaining: 19.4s
14:	learn: 0.5976142	total: 753ms	remaining: 19.3s
15:	learn: 0.5927864	total: 802ms	remaining: 19.3s
16:	learn: 0.5886408	total: 853ms	remaining: 19.2s
17:	learn: 0.5842163	total: 903ms	remaining: 19.2s
18:	learn: 0.5804251	total: 954ms	remaining: 19.1s
19:	learn: 0.5763854	total: 1s	remaining: 19.1s
20:	learn: 0.5728173	total: 1.05s	remaining: 19s
21:	learn: 0.5694814	total: 1.1s	remaining: 18.9s
22:	learn: 0.5663257	total: 1.15s	remaining: 18.9s
23:	learn: 0.5630638	total: 1.2s	remaining: 18.8s
24:	learn: 0.5600833	total: 1.25s	remaining: 18.8s
25:	learn: 0.5567336	total: 1.3s	remaining: 18.8s
26:	learn: 0.5537570	total: 1.36s	remaining: 18.7s
27:	learn: 0.5509210	total: 1.41s	remaining: 18.7s
28:	learn: 0.5483397	total: 1.46s	remaining: 18.7s
29:	learn: 0.5453628	total: 1.51s	remaining: 18.6s
30:	learn: 0.5428860	total: 1.56s	remaining: 18.6s
31:	learn: 0.5404179	total: 1.61s	remaining: 18.5s
32:	learn: 0.5380465	total: 1.66s	remai

174:	learn: 0.3993634	total: 10.9s	remaining: 14.1s
175:	learn: 0.3989628	total: 11s	remaining: 14s
176:	learn: 0.3985101	total: 11.1s	remaining: 14s
177:	learn: 0.3981082	total: 11.2s	remaining: 13.9s
178:	learn: 0.3975823	total: 11.2s	remaining: 13.9s
179:	learn: 0.3971543	total: 11.3s	remaining: 13.8s
180:	learn: 0.3966552	total: 11.4s	remaining: 13.8s
181:	learn: 0.3962919	total: 11.4s	remaining: 13.7s
182:	learn: 0.3959571	total: 11.5s	remaining: 13.7s
183:	learn: 0.3955892	total: 11.6s	remaining: 13.6s
184:	learn: 0.3952013	total: 11.7s	remaining: 13.6s
185:	learn: 0.3948199	total: 11.7s	remaining: 13.5s
186:	learn: 0.3945027	total: 11.8s	remaining: 13.4s
187:	learn: 0.3941252	total: 11.9s	remaining: 13.4s
188:	learn: 0.3937459	total: 12s	remaining: 13.3s
189:	learn: 0.3933742	total: 12s	remaining: 13.3s
190:	learn: 0.3929642	total: 12.1s	remaining: 13.2s
191:	learn: 0.3926053	total: 12.2s	remaining: 13.2s
192:	learn: 0.3922350	total: 12.2s	remaining: 13.1s
193:	learn: 0.3918746	

334:	learn: 0.3536466	total: 22.8s	remaining: 4.43s
335:	learn: 0.3534256	total: 22.9s	remaining: 4.36s
336:	learn: 0.3532261	total: 23s	remaining: 4.3s
337:	learn: 0.3530327	total: 23.1s	remaining: 4.23s
338:	learn: 0.3528522	total: 23.1s	remaining: 4.16s
339:	learn: 0.3526499	total: 23.2s	remaining: 4.1s
340:	learn: 0.3524875	total: 23.3s	remaining: 4.04s
341:	learn: 0.3522685	total: 23.4s	remaining: 3.97s
342:	learn: 0.3520840	total: 23.5s	remaining: 3.9s
343:	learn: 0.3518976	total: 23.6s	remaining: 3.83s
344:	learn: 0.3515993	total: 23.6s	remaining: 3.77s
345:	learn: 0.3514119	total: 23.7s	remaining: 3.7s
346:	learn: 0.3512098	total: 23.8s	remaining: 3.63s
347:	learn: 0.3510262	total: 23.9s	remaining: 3.56s
348:	learn: 0.3508154	total: 23.9s	remaining: 3.5s
349:	learn: 0.3506356	total: 24s	remaining: 3.43s
350:	learn: 0.3504497	total: 24.1s	remaining: 3.36s
351:	learn: 0.3502585	total: 24.1s	remaining: 3.29s
352:	learn: 0.3500708	total: 24.2s	remaining: 3.22s
353:	learn: 0.3498820

In [36]:

Stack_final_X_train = np.concatenate(( cat_train, lgbm_train), axis=1)
Stack_final_X_test = np.concatenate(( cat_test, lgbm_test), axis=1)
print('원본 학습 피처 데이터 Shape:',x_train.shape, '원본 테스트 피처 Shape:',test.shape)
print('스태킹 학습 피처 데이터 Shape:', Stack_final_X_train.shape,
      '스태킹 테스트 피처 데이터 Shape:',Stack_final_X_test.shape)

원본 학습 피처 데이터 Shape: (472432, 188) 원본 테스트 피처 Shape: (506691, 57)
스태킹 학습 피처 데이터 Shape: (1139754, 2) 스태킹 테스트 피처 데이터 Shape: (506691, 2)


In [37]:
from sklearn.linear_model import LogisticRegression
lr_final = LogisticRegression()
lr_final.fit(Stack_final_X_train, y_over)
stack_final = lr_final.predict_proba(Stack_final_X_test)[:,1]

In [38]:
stack_final

array([0.11678084, 0.07979786, 0.13080863, ..., 0.06298826, 0.08198328,
       0.34176441])

In [39]:
submission = pd.read_csv('/Users/krc/IEEE-CIS_Fraud_Detection/sample_submission.csv')

In [40]:
submission['isFraud']=stack_final

In [41]:
submission.head(3)

,TransactionID,isFraud
0,3663549,0.116781
1,3663550,0.079798
2,3663551,0.130809


In [42]:
submission.to_csv('year_dream2.csv',index=False)